<a href="https://colab.research.google.com/github/fanbyprinciple/kaggle_kernels/blob/master/fine_tuning_a_translation_system_for_bengali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from : https://kurianbenoy.com/ml-blog/fastai/huggingface/translation/fine%20tuning/malayalam/2022/03/13/hfenml-translate.html

The requirements were taken as it is from the blog. 

"It's built using Blurr library - built on top of Hugging face and fast.ai made by Wayde Gilliam. Also our translation system is going to be fine tuned on top of KDE specific dataset. You can find the trained model here."

In [1]:
! python3 -m pip install datasets 
! python3 -m pip install transformers[sentencepiece] 
! python3 -m pip install -Uqq ohmeow-blurr
! python3 -m pip install -U nltk 
! python3 -m pip install sacrebleu 

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 136 kB 37.7 MB/s 
     |████████████████████████████████| 77 kB 4.0 MB/s 
     |████████████████████████████████| 212 kB 40.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.9 MB/s 
     |████████████████████████████████| 127 kB 40.9 MB/s 
     |████████████████████████████████| 144 kB 44.5 MB/s 
     |████████████████████████████████| 271 kB 48.3 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.4 MB/s 


"We are using KDE4 datasets, and choose both source language and translation language as english and malayalam respectively. Usually these datasets are curated by community volunteers to their native language, and this was probably done by KDE community volunteers in Kerala. When someone is localizing these texts into there in local languague, usually computer science specific terms are still written in english." - from the orginal script

of course we will be trying out bengali.

In [2]:
import pandas
from datasets import load_dataset 

In [3]:
raw_dataset = load_dataset("kde4", lang1="en", lang2="ml", split="train[:1000]")

Using custom data configuration en-ml-lang1=en,lang2=ml


Generating train split: 0 examples [00:00, ? examples/s]

Dataset kde4 downloaded and prepared to /root/.cache/huggingface/datasets/kde4/en-ml-lang1=en,lang2=ml/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


In [4]:
raw_dataset[0]

{'id': '0',
 'translation': {'en': 'Add Feed to Akregator',
  'ml': 'അക്രിഗേറ്ററില്\u200d ഫീഡ് കൂട്ടിച്ചേര്\u200dക്കുക'}}

In [5]:
!pip install blurr

     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 132 kB 11.5 MB/s 
     |████████████████████████████████| 8.7 MB 38.6 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 


In [6]:

from fastai.data.all import *
from fastai.callback.all import *
from fastai.learner import *
from fastai.optimizer import *
from transformers import *

In [7]:
# !pip install transformers==3.5.1 
# !pip install ohmeow-blurr==0.0.18

In [8]:
from blurr.data.all import *
from blurr.modeling.all import *

ModuleNotFoundError: ignored

In [ ]:
pretrained_model_name = "Helsinki-NLP/opus-mt-en-ml"
model_cls = AutoModelForSeq2SeqLM
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)

In [ ]:
translation_df = pd.DataFrame(raw_datasets["translation"], columns=["en", "ml"])
translation_df.head()

In [ ]:
blocks = (Seq2SeqTextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader("en"), get_y=ColReader("ml"), splitter=RandomSplitter())

In [ ]:
dls = dblock.dataloaders(translation_df, bs=1)
dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=100, target_trunc_at=250)

In [ ]:
learn = BlearnerForTranslation.from_data(
    translation_df,
    pretrained_model_name,
    src_lang_name="English",
    src_lang_attr="en",
    trg_lang_name="Malayalam",
    trg_lang_attr="ml",
    dl_kwargs={"bs": 4},
)

In [ ]:
metrics_cb = BlearnerForTranslation.get_metrics_cb()
learn.fit_one_cycle(1, lr_max=4e-5, cbs=[metrics_cb])

In [ ]:
learn.show_results(learner=learn, input_trunc_at=500, target_trunc_at=250)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]["input_ids"].shape, b[1].shape

In [ ]:
dls.show_batch(dataloaders=dls, input_trunc_at=250, target_trunc_at=250)

In [ ]:
seq2seq_metrics = {"bleu": {"returns": "bleu"}, "meteor": {"returns": "meteor"}, "sacrebleu": {"returns": "score"}}

model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(
    dls,
    model,
    opt_func=partial(Adam),
    loss_func=PreCalculatedCrossEntropyLoss(),  # CrossEntropyLossFlat()
    cbs=learn_cbs,
    splitter=partial(blurr_seq2seq_splitter, arch=hf_arch),
)

learn.freeze()

In [ ]:
learn.lr_find(suggest_funcs=[minimum, steep, valley, slide])

In [ ]:
learn.fit_one_cycle(3, lr_max=0.0003981071640737355, cbs=fit_cbs)

In [ ]:
learn.show_results(learner=learn, input_trunc_at=500, target_trunc_at=500)

In [ ]:
test_text = "How are you doing"


outputs = learn.blurr_generate(test_text, key="translation_texts", num_return_sequences=3)
outputs

In [ ]:
export_fname = "saved_model"


learn.metrics = None
learn.export(fname=f"{export_fname}.pkl")

In [ ]:
test_text = "How are you doing"

inf_learn = load_learner(fname=f"{export_fname}.pkl")
inf_learn.blurr_translate(test_text)

In [ ]:
test_text1 = "Add All Found Feeds to Akregator."

inf_learn = load_learner(fname=f"{export_fname}.pkl")
inf_learn.blurr_translate(test_text1)


In [ ]:
test_text2 = "Subscribe to site updates (using news feed)."

inf_learn = load_learner(fname=f"{export_fname}.pkl")
inf_learn.blurr_translate(test_text2)